In [3]:
import os
import pickle
import torch
import datetime
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

# device = 'cuda:0'
device = 0
autovc_model = '/home/ytang363/7100_voiceConversion/pretrain/autovc.ckpt'
uttr = '002'
baseline = False

# load in models
if baseline:
    G = Generator(32,256,512,32).eval().to(device) # autovc.ckpt model shape
    autovc_model = '/home/ytang363/7100_voiceConversion/pretrain/autovc.ckpt'
else:
    G = Generator(16,256,512,16).eval().to(device) # oct-16 model shape
    autovc_model = '/home/ytang363/7100_voiceConversion/pretrain/oct-16_checkpoint_1500000.ckpt'

g_checkpoint = torch.load(autovc_model, map_location=torch.device(device))
print(g_checkpoint.keys())
G.load_state_dict(g_checkpoint['model'])

# metadata contains [filename, speaker embedding, spectrogram]
metadata = pickle.load(open(f'/home/ytang363/7100_voiceConversion/metadata_{uttr}.pkl', "rb"))
spect_vc = []   

dict_keys(['model', 'optimizer'])


In [4]:
for sbmt_i in metadata:
    print(f'source speaker: {sbmt_i[0]}')

    x_org = sbmt_i[2] # spectrogram
    x_org, len_pad = pad_seq(x_org)

    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device) # spectrogram
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device) # speaker embedding
    
    for sbmt_j in metadata:
        print(f'target speaker: {sbmt_j[0]}')
        
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else: # excluding a padding portion
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        # print(spect_vc)

current_date = datetime.datetime.now()
formatted_date = current_date.strftime("%m-%d")
# outName = f'{formatted_date}_author_results.pkl'
# outName = f'{formatted_date}_baseline-results.pkl'

if baseline:
    outName = f'{uttr}_baseline-results.pkl'
else:
    outName = f'{uttr}_results.pkl'

with open(outName, 'wb') as handle:
    pickle.dump(spect_vc, handle)       

source speaker: p225
target speaker: p225
target speaker: p226
target speaker: p227
target speaker: p228
target speaker: p229
target speaker: p230
target speaker: p231
target speaker: p232
target speaker: p233
target speaker: p234
target speaker: p237
source speaker: p226
target speaker: p225
target speaker: p226
target speaker: p227
target speaker: p228
target speaker: p229
target speaker: p230
target speaker: p231
target speaker: p232
target speaker: p233
target speaker: p234
target speaker: p237
source speaker: p227
target speaker: p225
target speaker: p226
target speaker: p227
target speaker: p228
target speaker: p229
target speaker: p230
target speaker: p231
target speaker: p232
target speaker: p233
target speaker: p234
target speaker: p237
source speaker: p228
target speaker: p225
target speaker: p226
target speaker: p227
target speaker: p228
target speaker: p229
target speaker: p230
target speaker: p231
target speaker: p232
target speaker: p233
target speaker: p234
target speake

In [4]:
res = pickle.load(open('11-02_baseline-results.pkl', "rb"))
res[0][1].shape

(194, 80)